## Capstone Project: The Battle of Neighborhoods [Part - 02]

### Mission

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

A description of the data and how it will be used to solve the problem. (15 marks)

## Required Data


The data necessary to resolve the requested information is:

Input data:
1. The Hotel address and coordinates: latitude and longitude. This information is important since it will help to locate the places of leisure, entertainment and services located near the lodging hotel.

2. The places of leisure, entertainment and services that are located near the hotel.

Output data:

1. List or report of the places of leisure, entertainment and services located near the hotel.
2. Distance between the hotel and the places.
3. List by categories.
4. List by Rating.
5. Like List.
6. Maps displaying localized places.



In this section a description of the data and how it will be used to solve the problem is provided. 

## How to solve the Problem

The address of the hotel will be used as input data to obtain more data on the categories of venues requested by the tourist or group of tourists and using data preparation tools, data analysis and data visualization, I will be able to obtain the results desired and be able to deliver to the end user the information they will use.
- With the hotel Name I'll find the Latitud and Longitud in FourSquare: api.foursquare.com/v2/venues/search
- With The Latitud and Longitude I could find the venues by the categories that tourist wanted, and they have to be near or close to the Hotel: api.foursquare.com/v2/venues/explore? and move the results in a dataframe.
- Using the venues of dataframe I need to fill it up with the likes and rating, so I need to access to foursquare again to request that data and create two columns using: api.foursquare.com/v2/venues
- Once the dataframe is updated, I coud make the output data thet the user or tourist need.

## Libraries to use


In [1]:
%reset -f
import matplotlib.pyplot as plt
import numpy as np 
from pandasql import *
import pandas as pd
from pandas.io.json import json_normalize
# visualization Data
import folium
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
# to calcule distance two points
from geopy.distance import geodesic

import warnings
warnings.filterwarnings("ignore")

print("Libraries Loaded")

Libraries Loaded


## Data preparation

In [2]:
## Get latitude and longitude The Hotel Data 

#Foursquare Credentials to Acces API FourSquare
CLIENT_ID = 'YJNK5Q1DMJNELAVAABGL0JOXFPBMQIPKINN5QFFBC51SFOJ0' # your Foursquare ID
CLIENT_SECRET = 'OTGFLSHNAD3VBVEFJBC0HCZFJ4SP2EYR1DBNRCKPRFRQDEKE' # your Foursquare Secret
VERSION = '20180604'

name = "Hotel Pennsylvania" # Venue Name to search in FourSquare API
city = "Manhattan, NY" # City where We will find the Hotel
LIMIT = 1 # Limit the response to one item
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near={}&query={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        city,
        name,
        LIMIT)
datarequested = requests.get(url).json()
datahotel = datarequested['response']['venues'][0]

hid = datahotel['id']
haddress = datahotel['location']['address']
hlatitude = datahotel['location']['lat']
hlongitude = datahotel['location']['lng']
origin = (hlatitude, hlongitude)

In [5]:
#Setting the Categories the tourist are interested in
namesel = ['Rental Car Location', 'Bank', 'Nightclub', 'Theater', 'Movie Theater', 'National Park']

LIMIT = 50 
city = 'Manhattan, NY'

# Accesing foursquare to request venues related with the category 
locations = {}
for selection in namesel:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&near={}&limit={}&query={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        hlatitude,
        hlongitude,
        city,
        LIMIT,
        selection)
    locations[selection] = requests.get(url).json()

In [6]:
# Normalizing Data requesting to FourSquare
data_venues={}
for catid in namesel:
    venues = json_normalize(locations[catid]['response']['groups'][0]['items'])
    data_venues[catid] = venues[['venue.name', 'venue.id', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    data_venues[catid].columns = ['Name', 'ID', 'Address', 'Lat', 'Lng']

In [8]:
# Extracting Data needed to Dataframe

Datanyc = pd.DataFrame(columns=['Name', 'ID', 'Address', 'Lat', 'Lng','Category'])
for catid in namesel:
    Datatemp = pd.DataFrame(data_venues[catid], columns=['Name', 'ID', 'Address', 'Lat', 'Lng'])
    Datatemp['Category'] = catid
    Datanyc = Datanyc.append(Datatemp, ignore_index=True)

Datanyc.dropna(how='all')    #to drop if all values in the row are nan    
Datanyc = Datanyc[['Category','Name', 'ID', 'Address', 'Lat', 'Lng']]
Datanyc.reset_index(drop=True, inplace=True)

Datanyc.head(10)

,Category,Name,ID,Address,Lat,Lng
0,Rental Car Location,Enterprise Rent-A-Car,4ad49a36f964a5204fe820e3,106 W 24th St,40.743671,-73.992900
1,Rental Car Location,Enterprise Rent-A-Car,4b60f29bf964a520fb022ae3,667 11th Ave,40.765045,-73.995458
2,Rental Car Location,Enterprise Rent-A-Car,4ba7e70cf964a52045bd39e3,403 E 65th St,40.763286,-73.958469
3,Rental Car Location,Hertz,4ba4d846f964a5208dba38e3,210 W 77th St,40.782047,-73.979798
4,Rental Car Location,Hertz,4bffd279f326ef3bcc2d0604,337 East 64th Street,40.763184,-73.960190
5,Rental Car Location,National Car Rental,4c28ab99d26eb713e64914d1,219 W 77th St,40.782299,-73.980380
6,Rental Car Location,National Car Rental,4e1591fbb61c42e7c54bfba0,252 W 40th St,40.755978,-73.990261
7,Rental Car Location,National Car Rental,4db3202293a017099dbcd1b1,138 E 50th St #146,40.756193,-73.971814
8,Rental Car Location,National Car Rental,5646c3dc498eec5cc3dbbac7,320 W 57th St,40.766881,-73.983699
9,Rental Car Location,National Car Rental,4bf5bddc9abec9b6c19624e8,142 E 31st St,40.744177,-73.980199


In [9]:
# showing and checking the categories that the user wanted is included in dataframe
print(Datanyc.groupby(["Category"])[['Category']].count())

                     Category
Category                     
Bank                       50
Movie Theater              50
National Park              12
Nightclub                  50
Rental Car Location        50
Theater                    50


In [11]:
#Calculating The distance. Input data: Lat and Lng of venues and the Lat and lng of the hotel 
Datanyc['Distance'] = ''
for index, row in Datanyc.iterrows():
    destine = (row["Lat"] ,  row["Lng"])
    distance = round(geodesic(origin, destine).miles, 2)
    Datanyc['Distance'].values[index] = distance
    
Datanyc = Datanyc[['Category','Name', 'ID', 'Address', 'Lat', 'Lng', 'Distance']]
Datanyc.sort_values(by=['Distance'], inplace=True) 
Datanyc.reset_index(drop=True, inplace=True)
Datanyc.head(10)

,Category,Name,ID,Address,Lat,Lng,Distance
0,Bank,PNC Bank,4cbf6aaa8db3b60c0a337878,"11 Penn Plz, 7th & W 31st St",40.749092,-73.991528,0.06
1,Rental Car Location,Avis Car Rental,4b7aa259f964a5209a342fe3,220 West 31st Street,40.749555,-73.993390,0.13
2,Bank,Chase Bank,4c0182fef7ab0f47024316b6,1260 Broadway,40.748416,-73.987626,0.21
3,Movie Theater,AMC 34th Street 14,45b893e3f964a520cf411fe3,312 W 34th St,40.752479,-73.994403,0.25
4,Rental Car Location,Hertz,4c083dd37e3fc928040bf082,323 West 34th Street,40.752821,-73.994522,0.27
5,Rental Car Location,U-Haul Neighborhood Dealer,5263df76498ebb8d61ce2f86,629 W 50th Street,40.748627,-73.996131,0.28
6,Nightclub,Liberty Hall at Ace Hotel New York,4b65fd87f964a520f00c2be3,20 W 29th St,40.745838,-73.988097,0.32
7,Nightclub,230 Fifth Rooftop Lounge,4459f9bff964a520e1321fe3,230 5th Ave,40.744232,-73.988660,0.41
8,Theater,Nederlander Theatre,4ef22ce4722eb49f525d2b06,208 W 41st St,40.755708,-73.988386,0.42
9,Rental Car Location,National Car Rental,4e1591fbb61c42e7c54bfba0,252 W 40th St,40.755978,-73.990261,0.42


In [12]:
#Creating two new columns for Likes and Rating 
Datanyc['Likes'] = ''
Datanyc['Rating'] = ''

In [13]:
Datanyc.head(10)

,Category,Name,ID,Address,Lat,Lng,Distance,Likes,Rating
0,Bank,PNC Bank,4cbf6aaa8db3b60c0a337878,"11 Penn Plz, 7th & W 31st St",40.749092,-73.991528,0.06,,
1,Rental Car Location,Avis Car Rental,4b7aa259f964a5209a342fe3,220 West 31st Street,40.749555,-73.993390,0.13,,
2,Bank,Chase Bank,4c0182fef7ab0f47024316b6,1260 Broadway,40.748416,-73.987626,0.21,,
3,Movie Theater,AMC 34th Street 14,45b893e3f964a520cf411fe3,312 W 34th St,40.752479,-73.994403,0.25,,
4,Rental Car Location,Hertz,4c083dd37e3fc928040bf082,323 West 34th Street,40.752821,-73.994522,0.27,,
5,Rental Car Location,U-Haul Neighborhood Dealer,5263df76498ebb8d61ce2f86,629 W 50th Street,40.748627,-73.996131,0.28,,
6,Nightclub,Liberty Hall at Ace Hotel New York,4b65fd87f964a520f00c2be3,20 W 29th St,40.745838,-73.988097,0.32,,
7,Nightclub,230 Fifth Rooftop Lounge,4459f9bff964a520e1321fe3,230 5th Ave,40.744232,-73.988660,0.41,,
8,Theater,Nederlander Theatre,4ef22ce4722eb49f525d2b06,208 W 41st St,40.755708,-73.988386,0.42,,
9,Rental Car Location,National Car Rental,4e1591fbb61c42e7c54bfba0,252 W 40th St,40.755978,-73.990261,0.42,,


In [150]:
# Access FourSquare to extracc Likes and Rating of the Venues I have in Dataframe and fill it up with that information
Rows = Datanyc.shape[0]
responses = []
for index in range(Rows): 
    id = Datanyc.loc[index,"ID"]
    venueid = id
    print ('Attempting to get information for venue with id = ', id)

    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(venueid,CLIENT_ID,CLIENT_SECRET,VERSION)    
    response = requests.get(url)
    
    if response.status_code != 200:
        print ('Failure response from Foursquare API, breaking...')
        break
    else:
        result = response.json()['response']
        responses.append(result)
        try:
            Datanyc.loc[index,"Likes"]=result['venue']['likes']['count']
        except:
            Datanyc.loc[index,"Likes"]=0
        try:
            Datanyc.loc[index,"Rating"]=result['venue']['rating']
        except:
            Datanyc.loc[index,"Rating"]="N/A"

print("Data Retrieval Complete")

Attempting to get information for venue with id =  581d85abbda2d737777605ec
Attempting to get information for venue with id =  3fd66200f964a520f5ea1ee3
Attempting to get information for venue with id =  5cd5d12bc8b2fb002c6a806c
Attempting to get information for venue with id =  544981d1498e6a33b8ab98f2
Attempting to get information for venue with id =  3fd66200f964a5206be31ee3
Attempting to get information for venue with id =  42893400f964a52054231fe3
Attempting to get information for venue with id =  5a35f6ae23472452931acac0
Attempting to get information for venue with id =  4df67e70e4cd50071441065f
Attempting to get information for venue with id =  507c50a3f1bae4f4c27a4e70
Attempting to get information for venue with id =  4c6d9b7b4d24b60cd65cd6d8
Attempting to get information for venue with id =  4aaa38e9f964a520a15520e3
Attempting to get information for venue with id =  4b5b579bf964a52014f628e3
Data Retrieval Complete


In [16]:
Datanyc.head(10) # showing data to verify if the likes and rating has been updated

,Category,Name,ID,Address,Lat,Lng,Distance,Likes,Rating
0,Bank,PNC Bank,4cbf6aaa8db3b60c0a337878,"11 Penn Plz, 7th & W 31st St",40.749092,-73.991528,0.06,6,6.6
1,Rental Car Location,Avis Car Rental,4b7aa259f964a5209a342fe3,220 West 31st Street,40.749555,-73.993390,0.13,1,6.5
2,Bank,Chase Bank,4c0182fef7ab0f47024316b6,1260 Broadway,40.748416,-73.987626,0.21,5,6.1
3,Movie Theater,AMC 34th Street 14,45b893e3f964a520cf411fe3,312 W 34th St,40.752479,-73.994403,0.25,1253,6.5
4,Rental Car Location,Hertz,4c083dd37e3fc928040bf082,323 West 34th Street,40.752821,-73.994522,0.27,5,5.7
5,Rental Car Location,U-Haul Neighborhood Dealer,5263df76498ebb8d61ce2f86,629 W 50th Street,40.748627,-73.996131,0.28,1,6.5
6,Nightclub,Liberty Hall at Ace Hotel New York,4b65fd87f964a520f00c2be3,20 W 29th St,40.745838,-73.988097,0.32,25,7.0
7,Nightclub,230 Fifth Rooftop Lounge,4459f9bff964a520e1321fe3,230 5th Ave,40.744232,-73.988660,0.41,2759,8.0
8,Theater,Nederlander Theatre,4ef22ce4722eb49f525d2b06,208 W 41st St,40.755708,-73.988386,0.42,220,8.9
9,Rental Car Location,National Car Rental,4e1591fbb61c42e7c54bfba0,252 W 40th St,40.755978,-73.990261,0.42,1,6.9
